In [ ]:
!pip install --upgrade google-ads

In [ ]:
import os
import json
import pickle

import google.ads.googleads.client
import google.protobuf.json_format

if __name__ == "__main__":
  os.environ["GOOGLE_ADS_CONFIGURATION_FILE_PATH"] = "/content/drive/MyDrive/Colab Notebooks/google-ads.yaml"

  directory_search_volume = '/content/drive/MyDrive/Colab Notebooks/search_volume/'

  files_all = os.listdir(directory_search_volume)
  files_keyword = [f for f in files_all if f.startswith('keywords_')]

  for file_keyword in files_keyword:

    with open(directory_search_volume + file_keyword, 'r') as file_keywords:
      keywords = file_keywords.read().splitlines()

      client_google_ads = google.ads.googleads.client.GoogleAdsClient.load_from_storage(version="v15")

      service_google_ads = client_google_ads.get_service("GoogleAdsService")
      service_keyword_plan_idea = client_google_ads.get_service("KeywordPlanIdeaService")

      request_generate_keyword_historical_metrics = client_google_ads.get_type("GenerateKeywordHistoricalMetricsRequest")
      request_generate_keyword_historical_metrics.customer_id = '1234567'
      request_generate_keyword_historical_metrics.keywords = keywords
      request_generate_keyword_historical_metrics.geo_target_constants.append(service_google_ads.geo_target_constant_path("2792"))
      request_generate_keyword_historical_metrics.keyword_plan_network = (client_google_ads.enums.KeywordPlanNetworkEnum.GOOGLE_SEARCH)
      request_generate_keyword_historical_metrics.language = service_google_ads.language_constant_path("1037")

      response_generate_keyword_historical_metrics = service_keyword_plan_idea.generate_keyword_historical_metrics(request = request_generate_keyword_historical_metrics)
      response_generate_keyword_historical_metrics_json = type(response_generate_keyword_historical_metrics).to_json(response_generate_keyword_historical_metrics)

      with open(directory_search_volume + 'pickle_' + file_keyword, 'wb') as file:
        pickle.dump(response_generate_keyword_historical_metrics_json, file)


In [ ]:
import pandas

with open('/content/drive/MyDrive/Colab Notebooks/search_volume/pickle_keywords_4', 'rb') as file:
  json_data = json.loads(pickle.load(file))

def extract_data_v3(entry):
    text = entry.get('text', None)
    close_variants = entry.get('closeVariants', [])
    monthly_search_volumes = entry.get('keywordMetrics', {}).get('monthlySearchVolumes', [])

    if not close_variants:
        close_variants = [text]

    extracted_data = []
    for volume in monthly_search_volumes:
        year = volume.get('year', None)
        month = volume.get('month', None)
        search_volume = volume.get('monthlySearches', None)
        date = f"{year}-{month:02d}" if year and month else None

        for variant in close_variants:
            if variant and date and search_volume is not None:
                extracted_data.append({'google_keyword': text, 'date': date, 'search_volume': search_volume, 'entered_keyword': variant})
    return extracted_data

extracted_data_v3 = []
for entry in json_data.get('results', []):
    extracted_data_v3.extend(extract_data_v3(entry))

df_v3 = pandas.DataFrame(extracted_data_v3)

In [ ]:
df_v3.to_excel('/content/drive/MyDrive/Colab Notebooks/search_volume/search_volume_4.xlsx', index=False)